In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os
import sounddevice as sd
import soundfile as sf
import threading
import time
import subprocess
import asyncio
import functools
import tempfile
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor

# Initialize recognizer
recognizer = sr.Recognizer()

# Global flags for mode switching
in_cmd_mode = False
assistant_active = False

# Define your wake words
wake_words = ["luna", "danna"]

# Simple cache for responses
response_cache = {}

# Create a temporary directory for audio files
temp_dir = tempfile.mkdtemp()

# Create a ThreadPoolExecutor
executor = ThreadPoolExecutor()

async def run_in_executor(func, *args):
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(executor, func, *args)

async def listen():
    global in_cmd_mode, assistant_active
    while True:
        if not in_cmd_mode:
            with sr.Microphone() as source:
                if not assistant_active:
                    print("Listening for wake word...")
                else:
                    print("Listening for commands...")
                audio = recognizer.listen(source)
                try:
                    command = await run_in_executor(recognizer.recognize_google, audio)
                    command = command.lower()
                    print(f"Command received: {command}")
                    if not assistant_active:
                        if any(wake_word in command for wake_word in wake_words):
                            assistant_active = True
                            await greet()
                    else:
                        if command in ["command", "cmd"]:
                            in_cmd_mode = True
                            print("Switched to command mode")
                        else:
                            await process_voice_command(command)
                except sr.UnknownValueError:
                    print("Sorry, I did not understand that.")
                except sr.RequestError:
                    print("Sorry, there was an error with the request.")
        await asyncio.sleep(0.1)

async def greet():
    greeting_text = "Greeting, Eheb. How can I assist you, my friend?"
    print(greeting_text)
    await speak(greeting_text)

async def generate_response(prompt):
    if prompt in response_cache:
        return response_cache[prompt]
    
    try:
        # Run Mistral locally using ollama
        result = await run_in_executor(
            lambda: subprocess.run(
                ['ollama', 'run', 'mistral', prompt],
                capture_output=True,
                text=True,
                check=True,
                encoding='utf-8'
            )
        )
        response = result.stdout.strip()
        response_cache[prompt] = response
        return response
    except subprocess.CalledProcessError as e:
        print(f"Error running Mistral: {e}")
        return "Sorry, I couldn't generate a response."
    except UnicodeDecodeError as e:
        print(f"Error decoding Mistral output: {e}")
        return "Sorry, there was an error processing the response."

async def speak(text):
    print("Assistant:", text)
    tts = gTTS(text=text, lang='en', tld='co.uk')
    mp3_path = os.path.join(temp_dir, "response.mp3")
    wav_path = os.path.join(temp_dir, "response.wav")
    tts.save(mp3_path)
    
    # Convert MP3 to WAV
    ffmpeg_command = [
        "ffmpeg",
        "-loglevel", "error",
        "-i", mp3_path,
        "-acodec", "pcm_s16le",
        "-ar", "44100",
        "-ac", "1",
        wav_path,
        "-y"
    ]
    
    try:
        await run_in_executor(lambda: subprocess.run(ffmpeg_command, check=True, capture_output=True))
    except subprocess.CalledProcessError as e:
        print(f"Error during audio conversion: {e.stderr}")
        return
    
    # Read and play WAV file
    data, samplerate = await run_in_executor(sf.read, wav_path)
    await run_in_executor(sd.play, data, samplerate)
    await run_in_executor(sd.wait)
    
    # Clean up temporary files
    os.remove(mp3_path)
    os.remove(wav_path)
    
async def process_voice_command(command):
    response_text = await generate_response(command)
    if response_text:
        print(f"Processing voice command: {command}")
        await speak(response_text)
    else:
        print("No response text generated.")

async def handle_command_mode():
    global in_cmd_mode, assistant_active
    while True:
        if in_cmd_mode:
            cmd_input = await run_in_executor(input, "Command mode active. Type 'ex' to switch back to voice mode: ")
            if cmd_input.lower() == "ex":
                in_cmd_mode = False
                assistant_active = True  # Keep the assistant active when exiting command mode
                print("Switched back to listening mode")
            else:
                await process_voice_command(cmd_input)
        await asyncio.sleep(0.1)

async def main():
    listener_task = asyncio.create_task(listen())
    command_mode_task = asyncio.create_task(handle_command_mode())
    
    try:
        await asyncio.gather(listener_task, command_mode_task)
    finally:
        # Clean up the temporary directory
        for file in os.listdir(temp_dir):
            os.remove(os.path.join(temp_dir, file))
        os.rmdir(temp_dir)
        executor.shutdown()

nest_asyncio.apply()

# Create and run the main coroutine
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

Listening for wake word...
Command received: luna
Greeting, Eheb. How can I assist you, my friend?
Assistant: Greeting, Eheb. How can I assist you, my friend?
Listening for commands...
Command received: tell me a joke
Processing voice command: tell me a joke
Assistant: Why don't scientists trust atoms? Because they make up everything! (Or, why did the scientist go out with a boron atom? Because he couldn't find any carbon monoxide!) Enjoy your day!
Listening for commands...
Command received: send me another one
Processing voice command: send me another one
Assistant: Here's a riddle for you: I speak without a mouth and breathe without lungs. What am I?

Ponder on it, and let's see if you can guess the answer!
Listening for commands...
Sorry, I did not understand that.
Listening for commands...
Command received: what's the answer
Processing voice command: what's the answer
Assistant: I'd be happy to help, but I need a bit more information. Could you please specify what question or probl

Command mode active. Type 'ex' to switch back to voice mode:  tell me  a riddle  +  it's answer


Processing voice command: tell me  a riddle  +  it's answer
Assistant: Riddle: I speak without a mouth and breathe without lungs. What am I?

Answer: The wind (or air). It is said to 'speak' by causing sounds when passing through narrow openings, and 'breathes' by moving in and out of spaces.


Command mode active. Type 'ex' to switch back to voice mode:  ex


Switched back to listening mode
Listening for commands...
Command received: thank you
Processing voice command: thank you
Assistant: You're welcome! If there is anything else I can help you with, feel free to ask. Have a great day!
Listening for commands...
Command received: no thanks goodbye then
Processing voice command: no thanks goodbye then
Assistant: Goodbye! Have a wonderful day! If you ever need help or have any questions, feel free to reach out. I'm always here to assist! Stay awesome! 🌟✨
Listening for commands...
Sorry, I did not understand that.
Listening for commands...
Sorry, I did not understand that.
Listening for commands...
Sorry, I did not understand that.
Listening for commands...
